In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
      # print(os.path.join(dirname, filename))
  
        

# Any results you write to the current directory are saved as output.

In [ ]:
from fastai import *
from fastai.vision import *
%matplotlib inline

In [ ]:
path='../input/imet-2020-fgvc7/'

In [ ]:
train_df=pd.read_csv(path + 'train.csv')
print(train_df.head())


In [ ]:
test_df=pd.read_csv(path + 'sample_submission.csv')
test_df.head()

In [ ]:
tfms = get_transforms(max_lighting=0.1, max_zoom=1.05, max_warp=0.)


In [ ]:
np.random.seed(42)
train_src = (ImageList.from_csv(path, 'train.csv', folder='train', suffix='.png')
       .split_by_rand_pct(0.2)
       .label_from_df(label_delim=' '))

In [ ]:
train_data = (train_src.transform(tfms, size=128)
        .databunch(bs=256).normalize(imagenet_stats))

In [ ]:
train_data.show_batch(rows=5, figsize=(12,9))

In [ ]:
arch=models.resnet50

In [ ]:
# creating directories and copying the models to those directories
Path('/root/.cache/torch/checkpoints/').mkdir(exist_ok=True, parents=True)
!cp '../input/resnet-fastai/resnet50-19c8e357.pth' '/root/.cache/torch/checkpoints/resnet50-19c8e357.pth'

In [ ]:
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)
learn = cnn_learner(train_data, arch, metrics=[acc_02, f_score])

In [ ]:
learn.model_dir='/kaggle/working/'

In [ ]:
#learn.lr_find()
#learn.recorder.plot(suggestion=True)

In [ ]:
lr=3.31E-02

In [ ]:
learn.fit_one_cycle(6, slice(lr))
learn.save('stage_1')
                    

In [ ]:
learn.load('stage_1')
#learn.save('stage_1')


In [ ]:
learn.path = Path('/kaggle/working')
learn.export()

In [ ]:
test = ImageList.from_folder(path+'test')
len(test)

In [ ]:
learn = load_learner(Path('/kaggle/working'), test=test)
preds, _ = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
thresh = 0.1
l_preds = [' '.join([learn.data.classes[i] for i,p in enumerate(pred) if p > thresh]) for pred in preds]
filenames = [f.name[:-4] for f in learn.data.test_ds.items]

In [ ]:
df = pd.DataFrame({'id':filenames, 'attribute_ids':l_preds},
                           columns=['id', 'attribute_ids'])
df.to_csv('submission.csv',index=False)

In [ ]:
d=pd.read_csv('submission.csv')
d.head()